In [1]:
!nvidia-smi

Thu Oct  3 08:41:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
%pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660544 sha256=0dc3c33e3eee947feca38568022df0151aec5fb04c0da647db683eec3900d9f0
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [14]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import time

# CUDA-код для суммы элементов вектора
vector_sum_kernel = """
__global__ void addKernel(int* result, int* a, unsigned int size) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    // Переменная для локальной суммы в блоке
    __shared__ int sharedSum[256];  // размер блока (можно изменить)

    int localSum = 0;

    // Суммируем элементы
    if (index < size) {
        localSum = a[index];
    }

    // Записываем локальную сумму в shared память
    sharedSum[threadIdx.x] = localSum;
    __syncthreads();  // Синхронизация потоков

    // Выполнение редукции в shared памяти
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            sharedSum[threadIdx.x] += sharedSum[threadIdx.x + stride];
        }
        __syncthreads();
    }

    // Записываем результат блока в глобальную память
    if (threadIdx.x == 0) {
        atomicAdd(result, sharedSum[0]);  // Используем atomic для безопасного инкремента результата
    }
}
"""


# Функция для сложения элементов вектора на GPU с использованием CUDA
def vector_sum_gpu(vector):
    '''
    # Функция для сложения элементов вектора на GPU
    :param: vector - входной вектор
    :return: [
      answer - результат умножения,
    ]
    '''

    start_time = time.time()

    # Выделение памяти на GPU
    vector_gpu = cuda.mem_alloc(vector.nbytes)
    result_gpu = cuda.mem_alloc(np.int32().nbytes)  # Память под результат
    initial_value = np.array([0], dtype=np.int32)

    # Инициализация результата
    cuda.memcpy_htod(result_gpu, initial_value)

    # Копирование данных на GPU
    cuda.memcpy_htod(vector_gpu, vector)

    # Компиляция и загрузка CUDA-кода
    mod = SourceModule(vector_sum_kernel)
    vector_sum = mod.get_function("addKernel")

    # Определяем размеры блока и сетки для распараллеливания
    block_size = 256
    grid_size = (len(vector) + block_size - 1) // block_size

    # Запуск ядра на GPU
    vector_sum(result_gpu, vector_gpu, np.int32(len(vector)), block=(block_size, 1, 1), grid=(grid_size, 1))

    # Копирование результата с GPU на CPU
    result = np.empty(1, dtype=np.int32)
    cuda.memcpy_dtoh(result, result_gpu)

    end_time = time.time()

    return result[0], end_time - start_time


def vector_sum_cpu(vector):
    '''
    Функция для сложения элементов вектора на CPU
    :param: vector - входной вектор
    :return: [
      answer - результат умножения,
    ]
    '''
    answer = 0
    for elem in vector:
        answer += elem
    return answer


if __name__ == "__main__":
    # Генерация вектора с случайными значениями
    vector_size = 500000
    vector = np.random.randint(1, 10, size=vector_size, dtype=np.int32)

    # Сложение на CPU
    start_time_cpu = time.time()
    answer_cpu = vector_sum_cpu(vector)
    end_time_cpu = time.time()

    time_cpu = end_time_cpu - start_time_cpu
    print(f"Сумма на CPU: {answer_cpu}, Время выполнения на CPU: {time_cpu} секунд")

    # Сложение на GPU
    answer_gpu, time_gpu = vector_sum_gpu(vector)
    print(f"Сумма на GPU: {answer_gpu}, Время выполнения на GPU: {time_gpu} секунд")

Сумма на CPU: 2498779, Время выполнения на CPU: 0.07734036445617676 секунд
Сумма на GPU: 2498779, Время выполнения на GPU: 0.0021088123321533203 секунд
